In [ ]:
import numpy as np 
import pandas as pd 

In [ ]:
data=pd.read_csv('/content/preprocessed_ade.csv')

In [ ]:
data

,Unnamed: 0,observation,labels
0,12606,early phase ii semi double blind study of the ...,0
1,685,cessation of mtx therapy led to complete regre...,0
2,5127,cutaneous biopsy showed hyphae and round inclu...,0
3,3158,she was treated with acyclovir and subsequentl...,1
4,10685,cisplatin was substituted and the patient achi...,0
...,...,...,...
23511,982,ticlopidine induced aplastic anemia two new c...,1
23512,14487,the risks of sumatriptan administration in pat...,0
23513,4680,however the spectrum of hosts and clinical p...,0
23514,2588,antibiotic treatment should be continued,0


In [ ]:
import gensim
import os
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


In [ ]:
data

,Unnamed: 0,observation,labels
0,12606,early phase ii semi double blind study of the ...,0
1,685,cessation of mtx therapy led to complete regre...,0
2,5127,cutaneous biopsy showed hyphae and round inclu...,0
3,3158,she was treated with acyclovir and subsequentl...,1
4,10685,cisplatin was substituted and the patient achi...,0
...,...,...,...
23511,982,ticlopidine induced aplastic anemia two new c...,1
23512,14487,the risks of sumatriptan administration in pat...,0
23513,4680,however the spectrum of hosts and clinical p...,0
23514,2588,antibiotic treatment should be continued,0


In [ ]:
!wget -P /root/input/ -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2021-06-24 07:28:42--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.154.38
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.154.38|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘/root/input/GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  55.5MB/s    in 30s     

2021-06-24 07:29:13 (52.3 MB/s) - ‘/root/input/GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [ ]:
url="https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [ ]:
embeddings = gensim.models.KeyedVectors.load_word2vec_format(url, binary=True)

In [ ]:
docs_vectors = pd.DataFrame() # creating empty final dataframe
stopwords = nltk.corpus.stopwords.words('english') # removing stop words
for doc in data['observation'].str.lower().str.replace('[^a-z ]', ''): # looping through each document and cleaning it
    temp = pd.DataFrame()  # creating a temporary dataframe(store value for 1st doc & for 2nd doc remove the details of 1st & proced through 2nd and so on..)
    for word in doc.split(' '): # looping through each word of a single document and spliting through space
        if word not in stopwords: # if word is not present in stopwords then (try)
            try:
                word_vec = embeddings[word] # if word is present in embeddings(goole provides weights associate with words(300)) then proceed
                temp = temp.append(pd.Series(word_vec), ignore_index = True) # if word is present then append it to temporary dataframe
            except:
                pass
    doc_vector = temp.mean() # take the average of each column(w0, w1, w2,........w300)
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True) # append each document value to the final dataframe
docs_vectors.shape

(23516, 300)

In [ ]:
 docs_vectors['labels'] = data['labels']

In [ ]:
docs_vectors = docs_vectors.dropna()

In [ ]:
docs_vectors

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,labels
0,-0.050695,-0.013357,-0.008664,0.066528,-0.052307,-0.042175,0.077194,-0.106771,0.094014,0.025045,-0.002279,-0.073015,-0.062754,0.023458,-0.113424,0.005196,-0.003326,0.046899,0.092621,-0.086620,-0.030538,-0.061499,-0.107239,-0.073883,-0.008952,0.003123,-0.046076,0.072083,0.063965,0.034098,-0.001490,-0.015025,0.024556,-0.051412,-0.079405,-0.059469,0.018873,-0.052607,0.112142,-0.071477,...,-0.039042,-0.066981,0.039108,0.031909,0.104273,-0.026176,-0.130332,-0.123133,0.029694,0.054385,0.017690,0.007355,-0.070262,0.055140,0.007670,-0.037689,-0.070933,-0.019201,0.056788,0.044965,0.039923,0.023499,0.012102,0.052272,-0.067769,0.005969,-0.063703,0.014497,0.013255,-0.005880,0.069450,-0.066691,-0.055615,-0.068281,-0.005549,-0.106692,0.031687,-0.025411,0.035197,0
1,-0.003021,0.127523,0.022970,0.078756,-0.108480,-0.006226,-0.025464,-0.258464,0.102407,-0.010974,-0.041590,-0.183228,0.019145,0.065308,0.030192,0.245097,0.080068,-0.014318,0.003296,-0.170492,0.016622,0.125977,-0.055054,0.191996,-0.136230,0.046834,-0.257161,-0.001546,0.073795,-0.008087,0.038411,-0.253988,-0.172902,0.072103,-0.274129,-0.037720,-0.071818,-0.001750,0.019694,-0.058512,...,0.023560,-0.005086,-0.016195,0.122925,0.153564,0.092306,-0.107666,-0.094686,-0.038452,0.129679,0.189931,-0.090637,-0.015818,0.167480,0.069153,-0.120809,0.080282,0.039800,0.011545,0.092285,0.134562,0.054606,-0.062337,-0.047445,-0.135681,0.168050,0.019450,0.060410,-0.014730,-0.113851,0.076541,0.077482,-0.045329,-0.001383,-0.022359,-0.056244,0.130605,0.129150,0.039032,0
2,-0.029696,0.117454,0.070279,0.049297,-0.143677,0.020880,-0.034909,-0.225763,0.071650,0.107422,-0.133434,-0.185225,-0.064337,0.098145,-0.072820,0.187689,0.063008,0.230735,-0.073231,0.010345,-0.069647,-0.056596,-0.062522,0.020130,-0.188754,-0.137140,-0.148304,0.077767,-0.025036,-0.118508,-0.092396,0.002475,0.010717,-0.007025,-0.138278,0.005085,0.043518,0.064731,0.043790,0.038613,...,0.048273,-0.010312,-0.112371,0.046531,0.120449,0.060059,-0.127644,0.040558,-0.075150,-0.036205,0.173329,-0.076527,-0.005085,0.192204,-0.018815,-0.092661,0.006947,0.019054,0.138184,0.096868,0.022367,0.050756,-0.051164,-0.029707,-0.083724,0.039218,-0.012851,-0.146107,0.018256,-0.018488,-0.023948,0.089930,0.033869,-0.042969,0.057861,-0.091569,-0.049128,0.084306,0.066426,0
3,-0.034995,0.044120,0.062256,0.079346,-0.163993,-0.040091,-0.057486,-0.120919,0.014265,0.075091,-0.192383,-0.232945,0.041138,0.021345,-0.151341,0.085903,-0.082441,0.236258,0.020050,0.091936,0.004220,0.075405,0.031786,-0.028285,-0.077676,-0.140102,-0.061558,0.049909,-0.002860,0.038225,-0.122105,-0.236049,-0.023071,0.013157,-0.068324,0.045576,0.005197,0.019531,-0.029465,-0.038295,...,-0.009234,-0.005153,-0.133806,0.153460,0.087899,0.072405,-0.104771,-0.013811,0.003383,0.105730,0.146362,-0.050642,-0.025257,0.165414,-0.028983,-0.061070,0.136161,0.002921,0.026228,-0.047032,0.073661,0.012591,-0.096392,-0.066232,-0.169316,0.125610,0.111904,-0.083758,0.105756,-0.125488,0.016497,-0.001325,0.106097,0.092913,0.194301,-0.168597,0.067226,0.183340,-0.034546,1
4,0.015993,0.063070,-0.002374,0.145820,-0.082438,0.000461,0.064053,-0.192383,0.125637,0.018826,-0.172716,-0.213867,0.005659,0.001294,0.017849,0.249525,0.088019,0.018311,-0.027913,-0.134142,0.034424,0.093031,-0.089423,0.083279,-0.061584,-0.042616,-0.188856,-0.040602,0.014445,0.056342,0.032718,-0.126126,-0.226671,-0.081546,-0.199599,-0.025684,0.005062,0.018643,0.045939,-0.028630,...,-0.041456,-0.042101,-0.080397,0.130995,0.052138,0.089925,-0.214205,-0.121718,-0.065077,0.069255,0.155545,-0.040944,0.047689,0.095893,-0.055393,-0.195258,-0.037672,0.029673,0.011854,0.120734,0.132958,-0.021444,-0.061561,-0.047670,-0.111681,0.130615,-0.031660,0.005324,0.039551,-0.141086,0.047343,0.018546

In [ ]:
from sklearn.model_selection import train_test_split 

In [ ]:
x=docs_vectors.drop('labels' , axis=1)

In [ ]:
y=docs_vectors['labels']

In [ ]:
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2 , random_state=2021)

In [ ]:
zerocount=0
onecount=0
for i in y_train:
  if i == 0:
    zerocount+=1
  else:
    onecount+=1
print("no of negative sample in training class is:",zerocount)
print("no of positive sample in training class is:",onecount) 

no of negative sample in training class is: 13328
no of positive sample in training class is: 5478


**DECISION TREE**

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.tree import DecisionTreeClassifier


In [ ]:
dt_model= DecisionTreeClassifier(max_depth=5)

In [ ]:
dt_model.fit(x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
dt_pred=dt_model.predict(x_test)

In [ ]:
dt_pred

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
print(classification_report(y_test,dt_pred))

              precision    recall  f1-score   support

           0       0.82      0.81      0.81      3359
           1       0.53      0.55      0.54      1343

    accuracy                           0.73      4702
   macro avg       0.67      0.68      0.67      4702
weighted avg       0.73      0.73      0.73      4702



In [ ]:
print(confusion_matrix(y_test,dt_pred))

[[2709  650]
 [ 610  733]]


In [ ]:
#training error 
dt_pred_train = dt_model.predict(x_train)
accuracy_score(y_train,dt_pred_train )  


0.7340210571094331

**LOGISTIC REGRESSION** 

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log_model=LogisticRegression(max_iter=1500)

In [ ]:
log_model.fit(x_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
log_pred=log_model.predict(x_test)

In [ ]:
log_pred

array([0, 0, 0, ..., 0, 1, 0])

In [ ]:
print(classification_report(log_pred,y_test))

              precision    recall  f1-score   support

           0       0.91      0.83      0.87      3682
           1       0.53      0.69      0.60      1020

    accuracy                           0.80      4702
   macro avg       0.72      0.76      0.73      4702
weighted avg       0.82      0.80      0.81      4702



In [ ]:
print(confusion_matrix(log_pred,y_test))

[[3047  635]
 [ 312  708]]


In [ ]:
#calculate training error 
log_y_pred_train = log_model.predict(x_train) 
accuracy_score(y_train,log_y_pred_train) 

0.7994257151972775

**NAIVE BAYES MODELS**

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
gnb_model=GaussianNB()


In [ ]:
gnb_model.fit(x_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
gnb_pred=gnb_model.predict(x_test)

In [ ]:
print(classification_report(y_test,gnb_pred))

              precision    recall  f1-score   support

           0       0.90      0.70      0.79      3359
           1       0.52      0.81      0.63      1343

    accuracy                           0.73      4702
   macro avg       0.71      0.75      0.71      4702
weighted avg       0.79      0.73      0.74      4702



In [ ]:
print(confusion_matrix(y_test,gnb_pred))

[[2339 1020]
 [ 259 1084]]


In [ ]:
gnb_train_pred = gnb_model.predict(x_train)
accuracy_score(y_train,gnb_train_pred)

0.7184409231096459

**RANDOM FOREST CLASSIFIER**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc_model=RandomForestClassifier(n_estimators=1000,max_leaf_nodes=50)

In [ ]:
rfc_model.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=50, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
rfc_pred=rfc_model.predict(x_test)

In [ ]:
print(classification_report(y_test,rfc_pred))

              precision    recall  f1-score   support

           0       0.78      0.99      0.87      3359
           1       0.91      0.31      0.46      1343

    accuracy                           0.79      4702
   macro avg       0.85      0.65      0.67      4702
weighted avg       0.82      0.79      0.76      4702



In [ ]:
print(confusion_matrix(y_test,rfc_pred))

[[3320   39]
 [ 927  416]]


In [ ]:
#training error 
rfc_train = rfc_model.predict(x_train)
accuracy_score(y_train,rfc_train)

0.8006487291290014

**XG BOOST**

In [ ]:
pip install xgboost 

In [ ]:
from xgboost import XGBClassifier

In [ ]:
xgb_model=XGBClassifier()

In [ ]:
xgb_model.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
xgb_pred = xgb_model.predict(x_test)


In [ ]:
print(classification_report(y_test,xgb_pred))

              precision    recall  f1-score   support

           0       0.85      0.93      0.89      3359
           1       0.77      0.57      0.66      1343

    accuracy                           0.83      4702
   macro avg       0.81      0.75      0.77      4702
weighted avg       0.82      0.83      0.82      4702



In [ ]:
#training error 
xgb_pred_train = xgb_model.predict(x_train) 
accuracy_score(y_train,xgb_pred_train)  

0.8540891204934595

In [ ]:
print(confusion_matrix(y_test,xgb_pred))

[[3130  229]
 [ 572  771]]
